# Machine Learning II: Assignments 2
Leon Berghoff, Jiawei Li, Strahinja Trenkic, Otto Riess

In [2]:
import pandas as pd
from bank_mkt import import_dataset, split_dataset, transform
from sklearn.preprocessing import FunctionTransformer, StandardScaler, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

# Data Preparation

A retail bank has a customer call centre as one of its units, through which the bank communicates with potential new clients and offers term deposits. Term deposits are defined as a fixed-term investment that includes the deposit of money into an account at a financial institution. Such an instrument would generate revenue for the bank, hence the bank records the outcomes of these phone calls along side other data related to the person being called, the economic indicators and certain parameters of the previous contact with the given person. By analysing previous phone calls the bank would like to improve its telemarketing results. 

The Bank Marketing dataset was collected by Moro, Cortez, and Rita (2014) and ordered by date ranging from May 2008 to November 2010.

In [3]:
bank_mkt = import_dataset("BankMarketing.csv")

func_transformer = FunctionTransformer(transform, kw_args={"gen": ["days", "has_previous"]})

num_features = ["month",
                "day_of_week",
                "campaign",
                "previous",
                "emp.var.rate",
                "cons.price.idx",
                "cons.conf.idx",
                "euribor3m",
                "nr.employed"]
standard_scaler = ColumnTransformer([("scaler", StandardScaler(), num_features)], remainder="passthrough")

preprocessor = make_pipeline(func_transformer, standard_scaler)

X_train, y_train, X_test, y_test, *_ = split_dataset(bank_mkt, preprocessor, test_size=0.6)

# Metrics

# Hyperparameters

In [5]:
from sklearn.svm import SVC
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
poly_svm = SVC(kernel="poly", class_weight="balanced", random_state=42)

poly_distributions = {"C": loguniform(1e0, 1e3),
    "degree": loguniform(1e0, 1e3),
    "gamma": ["scale", "auto"],
    }

poly_search = RandomizedSearchCV(poly_svm,
    poly_distributions,
    scoring="average_precision",
    cv=5,
    n_jobs=-1,
    n_iter=16)

poly_fit = poly_search.fit(X_train, y_train)
poly_results = random_search.cv_results_
poly_best_params = random_search.best_params_
poly_best_score = random_search.best_score_

print(f"best parameters found: {poly_best_params}, with mean test score: {poly_best_score}")